In [1]:


from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
import zipfile
import os
from PIL import Image

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.svm import LinearSVC
from sklearn import svm
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score

In [2]:
from keras.applications.vgg16 import VGG16
model = VGG16()

553476096/553467096 [==============================] - 6s 0us/step


In [31]:
m_files = os.listdir('/content/drive/MyDrive/fashion-data/men')
w_files = os.listdir('/content/drive/MyDrive/fashion-data/women')

#merge both datasets 
all_files = m_files + w_files
print('total:',len(all_files))

total: 2512


In [39]:
#convert image files into arrays
path = '/content/drive/MyDrive/fashion-data'
def read_img(file,name,pth):
    images = np.zeros((14700))
    for image in file: 
        arr = Image.open(pth+'/'+name+'/'+image) #get img array 
        img = arr.resize((70,70)) #resize for standard sizing
        arr = np.asarray(img) #turn into array
        img.close()
        flatten = arr.flatten() #flatten
        images = np.vstack((images,flatten)) #stack 
    images = np.delete(images, 0, 0)
    return images

#read images from files 
men = read_img(m_files[:],'men',path)
women = read_img(w_files[:],'women',path)

#turn into dataframes and add class labels
men = pd.DataFrame(men)
men['label'] = 0
women = pd.DataFrame(women)
women['label'] = 1

#merge and shuffle
fashion = men.append(women,ignore_index=True)
fashion = shuffle(fashion)

#separate dependent and independent variables
X = fashion.loc[:, fashion.columns != 'label']
y = fashion['label']

#see dataframe
fashion.head()

#split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.50, random_state=42)

print('done baby')

In [45]:
##however 
#input array will need to be 4-dimensional: samples, rows, columns, and channels.

#prepare input for the network

(1000,)